In [1]:
from tqdm import tqdm
import math
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
enwik6 = open('enwik6', 'rb').read()

**Compression**

= Coding + Modelling

Coding: How to store symbols.  
Modelling: Which symbols are more probable.

We want to code the most probable symbols in the least amount of bits. And vice versa.

Coding is largely solved (arithmetic coding)  
Modelling is proven unsolvable.

In [3]:
enwik6_zip = open('enwik6.zip', 'rb').read()

In [4]:
len(enwik6) / len(enwik6_zip)

2.8007440895361713

**Baseline Compression factors (enwik6)**:  

| Model | bpc |
|-------|-------|
| Zip Compression | 2.8007440895361713 (actual) |
| Simple Byte Frequency | 1.320 (worst case) |
| byte tri gram | 1.710 (worst case) |
| starlit | 8.68 (on enwik9) |

Shannon and Weaver (1949) proved that the best you can do for a symbol with probability p is assign a code
of length $log_2 1/p$

Simple char frequency (0 order model)

In [5]:
freqs = defaultdict(int)
bits = 0
total = 0
for i in tqdm(enwik6):
    bits += 1 + math.log2(1 / ((freqs[i] + 1)/(total + 256)))  
    freqs[i] += 1
    total += 1

100%|████████████████████████████| 1004232/1004232 [00:00<00:00, 2054942.31it/s]


In [6]:
len(enwik6) / (bits/8)

1.3201943620319303

char level bi-gram

In [7]:
n = 1

In [8]:
freqs = defaultdict(lambda: defaultdict(int)) #freqs[context][char] = count
bits = 0
total = 0
context = enwik6[:n]
for i in tqdm(enwik6[n:]):
    prob = ((freqs[context][i] + 1) / (sum(freqs[context].values()) + 256))
#     print(freqs[context][i], prob)
    bits += 1 + math.log2(1 / prob)   
    freqs[context][i] += 1 
    context += bytes([i])
    context = context[-n:]


100%|█████████████████████████████| 1004231/1004231 [00:01<00:00, 720162.20it/s]


In [9]:
len(enwik6) / (bits/8)

1.6137634914266308

char level tri-gram

In [10]:
n = 2

In [11]:
freqs = defaultdict(lambda: defaultdict(int)) #freqs[context][char] = count
bits = 0
total = 0
context = enwik6[:n]
for i in tqdm(enwik6[n:]):
    prob = ((freqs[context][i] + 1) / (sum(freqs[context].values()) + 256))
#     print(freqs[context][i], prob)
    bits += 1 + math.log2(1 / prob)   
    freqs[context][i] += 1 
    context += bytes([i])
    context = context[-n:]


100%|█████████████████████████████| 1004230/1004230 [00:01<00:00, 779310.59it/s]


In [12]:
len(enwik6) / (bits/8)

1.7099700762813315

char level 4-gram

In [78]:
n = 3

In [79]:
freqs = defaultdict(lambda: defaultdict(int)) #freqs[context][char] = count
bits = 0
total = 0
context = enwik6[:n]
for i in tqdm(enwik6[n:]):
    prob = ((freqs[context][i] + 1) / (sum(freqs[context].values()) + 256))
#     print(freqs[context][i], prob)
    bits += 1 + math.log2(1 / prob)   
    freqs[context][i] += 1 
    context += bytes([i])
    context = context[-n:]

100%|█████████████████████████████| 1004229/1004229 [00:01<00:00, 711861.37it/s]


In [80]:
len(enwik6) / (bits/8)

1.5526526216737395

TODO: Backoff would be nice also the thing where all less n grams are used with some weights

In [ ]:
n = 2
freqs = defaultdict(lambda: defaultdict(int)) #freqs[context][char] = count
bits = 0
total = 0
context = ''
for i in tqdm(enwik6[n:]):
    prob = ((freqs[context][i] + 1) / (sum(freqs[context].values()) + 256))
    bits += 1 + math.log2(1 / prob)   
    freqs[context][i] += 1 
    context += bytes([i])
    context = context[-n:]

**Arithmetic Encoding**

In [109]:
n = 2

In [114]:
freqs = defaultdict(lambda: defaultdict(int)) #freqs[context][char] = count
bits = 0
total = 0
context = enwik6[:n]
lower = 0
upper = 1
for i in tqdm(enwik6[n:]):
    total = (sum(freqs[context].values()) + 256)
    prob = ((freqs[context][i] + 1) / total)
    bits += 1 + math.log2(1 / prob)   
    probs = [(freqs[context][x] + 1)/total for x in range(256)]
    assert sum(probs) == 1, f"sum of probs is {sum(probs)}"
    r = upper - lower
    assert r > 0, f"range is {r}"
    upper = lower + (r * sum(probs[:i]))
    lower = lower + (r * sum(probs[:i-1]))
    print(upper, lower)
    freqs[context][i] += 1 
    context += bytes([i])
    context = context[-n:]

  0%|                                     | 8/1004230 [00:00<03:33, 4698.84it/s]

0.39453125 0.390625
0.39215087890625 0.3921356201171875
0.3921418786048889 0.39214181900024414
0.3921418415848166 0.39214184135198593
0.3921418414602158 0.3921418414593063
0.39214184145967934 0.3921418414596758
0.3921418414596773 0.39214184145967723
0.39214184145967723 0.39214184145967723


AssertionError: range is 0.0